# Imports

In [225]:
import pickle
import pandas as pd
import sqlite3
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

# Loading data

In [226]:
data = '/home/uadson/repos/machine-learn/data/database/chatlocaldb.sqlite3'

In [227]:
conn = sqlite3.connect(data)

department_qs = "SELECT * FROM departments_department;"
services_qs = "SELECT * FROM services_service;"
questions_qs = "SELECT * FROM questions_question;"

department_df = pd.read_sql_query(department_qs, conn)
services_df = pd.read_sql_query(services_qs, conn)
questions_df = pd.read_sql_query(questions_qs, conn)

# Preprocessing

In [228]:
department_df.head()

,id,created_at,updated_at,active,name,category
0,1,2023-09-13 14:37:17,2023-09-13 14:37:28.602000,1,Secretaria Municipal De Finanças,Finanças


In [229]:
services_df.head()

,id,created_at,updated_at,active,name,index,department_id,profile_id
0,1,2023-09-15 15:04:26.549000,2024-01-16 13:26:17.796000,1,ENTENDA SEU IPTU,1,1,4
1,2,2023-09-15 15:14:38.771000,2023-09-15 15:14:38.775000,1,NOTA FISCAL,2,1,5
2,3,2023-09-15 15:32:25.555000,2024-01-24 10:06:59.069000,1,SIMPLES NACIONAL 2024,3,1,5
3,4,2023-09-15 15:42:50.880000,2023-09-18 13:36:32.251000,1,CADASTRO MOBILIÁRIO 1,4,1,5
4,5,2023-09-15 15:47:02.408000,2023-09-15 16:12:51.991000,1,ISS,5,1,5


In [230]:
questions_df.head()

,id,created_at,updated_at,active,title,answer,url,index,department_id,profile_id,service_id
0,1,2023-09-15 15:09:14.343000,2024-01-16 15:33:35.386000,1,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,https://www.goiania.go.gov.br/sefin/iptu/,1,1,4,1
1,2,2023-09-15 15:11:04.308000,2023-12-05 11:04:00.894000,1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,None,2,1,4,1
2,3,2023-09-15 15:12:06.972000,2024-01-16 14:00:26.230000,1,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",None,3,1,4,1
3,4,2023-09-15 15:12:54.616000,2024-01-16 14:02:04.196000,1,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",None,4,1,4,1
4,5,2023-09-15 15:15:50.990000,2023-12-04 13:17:34.529000,1,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,None,1,1,5,2


In [231]:
questions = questions_df["title answer department_id service_id".split(" ")]
questions.head()

,title,answer,department_id,service_id
0,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,1,1
1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,1,1
2,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",1,1
3,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",1,1
4,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,1,2


In [232]:
questions.columns = ["title answer department service".split(" ")]
questions.head()

,title,answer,department,service
0,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,1,1
1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,1,1
2,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",1,1
3,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",1,1
4,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,1,2


In [233]:
questions["department"] = questions["department"].replace(1, "Finanças")
questions.head()

/tmp/ipykernel_366951/2846663326.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["department"] = questions["department"].replace(1, "Finanças")


,title,answer,department,service
0,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,Finanças,1
1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,Finanças,1
2,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",Finanças,1
3,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",Finanças,1
4,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,Finanças,2


In [234]:
for question_col in range(len(questions["service"])):
    question_service_pk = questions.iloc[question_col, 3] # id da coluna service
    for service_col in range(len(services_df["id"])):
        service_pk = services_df.iloc[service_col, 0] # id da coluna id em service_df
        
        if question_service_pk == service_pk:
            questions["service"] = questions["service"].replace(question_service_pk, services_df.iloc[service_col, 4])

questions.head()

/tmp/ipykernel_366951/3801845715.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["service"] = questions["service"].replace(question_service_pk, services_df.iloc[service_col, 4])


,title,answer,department,service
0,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,Finanças,ENTENDA SEU IPTU
1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,Finanças,ENTENDA SEU IPTU
2,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",Finanças,ENTENDA SEU IPTU
3,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",Finanças,ENTENDA SEU IPTU
4,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,Finanças,NOTA FISCAL


In [235]:
questions = questions["department service title answer".split(" ")]

questions

,department,service,title,answer
0,Finanças,ENTENDA SEU IPTU,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...
1,Finanças,ENTENDA SEU IPTU,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...
2,Finanças,ENTENDA SEU IPTU,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do..."
3,Finanças,ENTENDA SEU IPTU,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc..."
4,Finanças,NOTA FISCAL,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...
...,...,...,...,...
129,Finanças,SIMPLES NACIONAL 2024,Quem deve solicitar opção pelo Simples Nacional?,Os contribuintes que foram excluídos por qualq...
130,Finanças,SIMPLES NACIONAL 2024,Qual o prazo para solicitar opção?,A opção deverá ser formalizada até o último di...
131,Finanças,SIMPLES NACIONAL 2024,"Uma vez solicitada, a opção pelo Simples Nacio...",Enquanto não vencido o prazo para formalização...
132,Finanças,SIMPLES NACIONAL 2024,"Após cancelada, poderá ser formalizada nova op...","Após cancelado o pedido, poderá ser formalizad..."


In [236]:
len(questions.columns)

4

In [237]:
X_questions = questions.iloc[:, 0:3].values

X_questions[0]

array(['Finanças', 'ENTENDA SEU IPTU',
       'Como emitir o boleto do IPTU 2024?'], dtype=object)

In [238]:
y_questions = questions.iloc[:, 3].values

y_questions

array(['O procedimento é simples. Após acessar o portal da Prefeitura, www.goiania.go.gov.br, o interessado tem que clicar no link “Débitos IPTU e ITU 2024”, na tela principal do site. Em seguida inserir o número da inscrição cadastral do proprietário do imóvel para visualizar o boleto e o valor a ser pago. A ferramenta ainda disponibiliza os dados cadastrais do imóvel para conferência dos valores.',
       'Em qualquer boleto de IPTU dos anos anteriores. Caso não encontre, o contribuinte pode agendar seu atendimento em uma das unidades do Atende Fácil pelo agendamento no site www.goiania.go.gov.br, apresentando documentos pessoais e do imóvel.',
       'Até o ano de 2025, o IPTU não terá reajuste do imposto, que sofrerá apenas correção inflacionária do período. Em 2024, o índice inflacionário é de 4,68%. São exceções à correção por inflação aqueles que tiveram acréscimo de área de terreno, edificação na área, alteração de residencial para não residencial, remanejamentos ou desmembrame

In [239]:
label_encoder_department = LabelEncoder().fit_transform(X_questions[:, 0])
label_encoder_service = LabelEncoder().fit_transform(X_questions[:, 1])
label_encoder_title = LabelEncoder().fit_transform(X_questions[:, 2])

In [240]:
X_questions[:, 0] = label_encoder_department
X_questions[:, 1] = label_encoder_service
X_questions[:, 2] = label_encoder_title

In [241]:
X_questions

array([[0, 2, 12],
       [0, 2, 65],
       [0, 2, 111],
       [0, 2, 123],
       [0, 7, 17],
       [0, 2, 26],
       [0, 7, 129],
       [0, 2, 2],
       [0, 7, 59],
       [0, 2, 94],
       [0, 7, 60],
       [0, 7, 131],
       [0, 7, 125],
       [0, 7, 45],
       [0, 7, 128],
       [0, 7, 42],
       [0, 12, 9],
       [0, 12, 85],
       [0, 12, 23],
       [0, 12, 22],
       [0, 12, 106],
       [0, 12, 1],
       [0, 12, 57],
       [0, 12, 58],
       [0, 12, 39],
       [0, 12, 126],
       [0, 2, 120],
       [0, 12, 95],
       [0, 12, 24],
       [0, 12, 3],
       [0, 12, 44],
       [0, 0, 29],
       [0, 0, 93],
       [0, 5, 69],
       [0, 2, 7],
       [0, 2, 51],
       [0, 2, 89],
       [0, 2, 11],
       [0, 11, 113],
       [0, 11, 72],
       [0, 11, 87],
       [0, 2, 88],
       [0, 2, 25],
       [0, 1, 13],
       [0, 6, 101],
       [0, 6, 37],
       [0, 6, 104],
       [0, 6, 91],
       [0, 6, 96],
       [0, 6, 118],
       [0, 6, 38],
      

In [242]:
onehotencoder_questions = ColumnTransformer(
    transformers=[
        (
            "OneHot",
            OneHotEncoder(),
            [0, 1, 2],
        ),
    ],
    remainder="passthrough"
)

In [243]:
X_questions = onehotencoder_questions.fit_transform(X_questions).toarray()

In [244]:
X_questions

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [245]:
X_questions[0]

array([1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [246]:
X_questions.shape

(134, 150)

In [247]:
scaler_questions = StandardScaler()

X_questions = scaler_questions.fit_transform(X_questions)

In [248]:
X_questions[0]

array([ 0.        , -0.12309149, -0.086711  ,  2.1408721 , -0.43109246,
       -0.1754116 , -0.21650635, -0.3939193 , -0.31362502, -0.086711  ,
       -0.086711  , -0.19687481, -0.1754116 , -0.44320263, -0.19687481,
       -0.25197632, -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , 11.53256259, -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.08

In [249]:
X_questions_train, X_questions_test, y_questions_train, y_questions_test = train_test_split(X_questions, y_questions, test_size=0.3, random_state=0)

In [250]:
X_questions_train.shape, X_questions_test.shape

((93, 150), (41, 150))

In [251]:
y_questions_train.shape, y_questions_test.shape

((93,), (41,))

In [252]:
with open("/home/uadson/repos/machine-learn/data/database/chatbot.pkl", mode="wb") as file:
    pickle.dump([X_questions_train, y_questions_train, X_questions_test, y_questions_test], file)